# Azure audio whisper (プレビュー) 例

> 注意: openaiライブラリの新しいバージョンが利用可能です。https://github.com/openai/openai-python/discussions/742 を参照してください

この例では、Azure OpenAI Whisperモデルを使用してオーディオファイルを転写する方法を示しています。

## セットアップ

まず、必要な依存関係をインストールします。

In [ ]:
! pip install "openai>=0.28.1,<1.0.0"
! pip install python-dotenv

次に、ライブラリをインポートし、Azure OpenAIサービスで動作するようにPython OpenAI SDKを設定します。

> 注意: この例では、コード内で変数を設定することでライブラリがAzure APIを使用するように設定しました。開発時には、代わりに環境変数を設定することを検討してください：

```
OPENAI_API_BASE
OPENAI_API_KEY
OPENAI_API_TYPE
OPENAI_API_VERSION
```

In [1]:
import os
import dotenv
import openai


dotenv.load_dotenv()

True

Azure OpenAI Serviceに適切にアクセスするには、[Azure Portal](https://portal.azure.com)で適切なリソースを作成する必要があります（これを行う方法の詳細なガイドは[Microsoft Docs](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal)で確認できます）。

リソースが作成されたら、最初に必要なのはそのエンドポイントです。エンドポイントは、*"Resource Management"*セクション内の*"Keys and Endpoints"*セクションで確認できます。これを取得したら、この情報を使用してSDKをセットアップします：

In [2]:
openai.api_base = os.environ["OPENAI_API_BASE"]

# Min API version that supports Whisper
openai.api_version = "2023-09-01-preview"

# Enter the deployment_id to use for the Whisper model
deployment_id = "<deployment-id-for-your-whisper-model>"

### 認証

Azure OpenAIサービスは、APIキーとAzure認証情報を含む複数の認証メカニズムをサポートしています。

In [3]:
# set to True if using Azure Active Directory authentication
use_azure_active_directory = False

#### APIキーを使用した認証

OpenAI SDKを*Azure APIキー*を使用するように設定するには、`api_type`を`azure`に設定し、`api_key`をエンドポイントに関連付けられたキーに設定する必要があります（このキーは[Azure Portal](https://portal.azure.com)の*「リソース管理」*の下の*「キーとエンドポイント」*で確認できます）

In [4]:
if not use_azure_active_directory:
    openai.api_type = 'azure'
    openai.api_key = os.environ["OPENAI_API_KEY"]

#### Azure Active Directoryを使用した認証
それでは、Microsoft Active Directory認証を通じてキーを取得する方法を見てみましょう。

In [ ]:
from azure.identity import DefaultAzureCredential

if use_azure_active_directory:
    default_credential = DefaultAzureCredential()
    token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

    openai.api_type = 'azure_ad'
    openai.api_key = token.token

トークンは一定期間有効で、その後期限切れになります。すべてのリクエストで有効なトークンが送信されるようにするには、`requests.auth`にフックして期限切れのトークンを更新することができます：

In [ ]:
import typing
import time
import requests

if typing.TYPE_CHECKING:
    from azure.core.credentials import TokenCredential

class TokenRefresh(requests.auth.AuthBase):

    def __init__(self, credential: "TokenCredential", scopes: typing.List[str]) -> None:
        self.credential = credential
        self.scopes = scopes
        self.cached_token: typing.Optional[str] = None

    def __call__(self, req):
        if not self.cached_token or self.cached_token.expires_on - time.time() < 300:
            self.cached_token = self.credential.get_token(*self.scopes)
        req.headers["Authorization"] = f"Bearer {self.cached_token.token}"
        return req

if use_azure_active_directory:
    session = requests.Session()
    session.auth = TokenRefresh(default_credential, ["https://cognitiveservices.azure.com/.default"])

    openai.requestssession = session

## 音声文字起こし

音声文字起こし（音声テキスト変換）は、話された言葉をテキストに変換するプロセスです。音声ファイルストリームをテキストに文字起こしするには、`openai.Audio.transcribe`メソッドを使用します。

サンプル音声ファイルは[GitHubのAzure AI Speech SDKリポジトリ](https://github.com/Azure-Samples/cognitive-services-speech-sdk/tree/master/sampledata/audiofiles)から取得できます。

In [9]:
# download sample audio file
import requests

sample_audio_url = "https://github.com/Azure-Samples/cognitive-services-speech-sdk/raw/master/sampledata/audiofiles/wikipediaOcelot.wav"
audio_file = requests.get(sample_audio_url)
with open("wikipediaOcelot.wav", "wb") as f:
    f.write(audio_file.content)

In [ ]:
transcription = openai.Audio.transcribe(
    file=open("wikipediaOcelot.wav", "rb"),
    model="whisper-1",
    deployment_id=deployment_id,
)
print(transcription.text)